In [1]:
import os
import sys
import time
import json
from os.path import abspath, dirname
import pandas as pd
import numpy as np
import datetime

# current_dir = abspath(dirname(__file__))
current_dir = abspath(dirname(""))
project_path = abspath(dirname(dirname(current_dir)))
sys.path.append(project_path + "/TradeStreamEngine/StreamEngine")

# current_dir = abspath(dirname(__file__))
current_dir = abspath(dirname(""))
project_path = abspath(dirname(dirname(current_dir)))
sys.path.append(project_path + "/TradeStreamEngine/StreamEngineBase")

import synthesis
import coins.btcflow as btcflow

In [2]:
class btcSynth():

    def __init__(self, level_size, exchanges = ["binance", "okx", "bybit", "bitget", "bingx", "mexc", "kucoin", "deribit", "coinbase", "htx", "gateio"]):
        self.exchanges = exchanges
        self.level_size = level_size
        self.axis = {
            "binance" : btcflow.binance_flow(level_size),
            "okx" : btcflow.okx_flow(level_size),
            "bybit" : btcflow.bybit_flow(level_size),
            "bingx" : btcflow.bingx_flow(level_size),
            "mexc" : btcflow.mexc_flow(level_size),
            "kucoin" : btcflow.kucoin_flow(level_size),
            "deribit" : btcflow.deribit_flow(level_size),
            "coinbase" : btcflow.coinbase_flow(level_size),
            "htx" : btcflow.htx_flow(level_size),
            "gateio" : btcflow.gateio_flow(level_size),
            "bitget" : btcflow.bitget_flow(level_size),
        }
        for exchange in self.axis.keys():
            if exchange not in self.exchanges:
                del self.axis[exchange]

        self.books = {
            "spot" : synthesis.booksmerger("btc", "spot", axis = {
                "_".join([exchange, flow]) : self.axis[exchange].spot_axis["books"][flow] for exchange in self.axis.keys() for flow in self.axis[exchange].spot_axis["books"].keys()
            }
                                           ),
            "perpetual" : synthesis.booksmerger("btc", "perpetul", axis = {
                "_".join([exchange, flow]) : self.axis[exchange].perpetual_axis["books"][flow] for exchange in self.axis.keys() for flow in self.axis[exchange].perpetual_axis["books"].keys()
            }
                                           ),
        }
        self.trades = {
            "spot" : synthesis.tradesmerger("btc", "spot", axis = {
                "_".join([exchange, flow]) : self.axis[exchange].spot_axis["trades"][flow] for exchange in self.axis.keys() if exchange != "deribit" for flow in self.axis[exchange].spot_axis["trades"].keys()
            }
                                           ),
            "perpetual" : synthesis.tradesmerger("btc", "perpetul", axis = {
                "_".join([exchange, flow]) : self.axis[exchange].perpetual_axis["trades"][flow] for exchange in self.axis.keys() if exchange != "coinbase" for flow in self.axis[exchange].perpetual_axis["trades"].keys() 
            }
                                           ),
        }

        self.adjustmests = {
            "spot" : synthesis.booksadjustments("btc", "spot", self.books["spot"], self.trades["spot"]),
            "perpetual" : synthesis.booksadjustments("btc", "perpetual", self.books["perpetual"], self.trades["perpetual"])
        }

        self.oifunding =  synthesis.oiomnifier("btc", "perpetul", axis = {
                "_".join([exchange, flow]) : self.axis[exchange].perpetual_axis["oifunding"][flow] for exchange in self.axis.keys() if exchange != "coinbase" for flow in self.axis[exchange].perpetual_axis["oifunding"].keys() 
            }
                                           )

        self.liquidations =  synthesis.lomnifier("btc", "perpetul", axis = {
                "_".join([exchange, flow]) : self.axis[exchange].perpetual_axis["liquidations"][flow] for exchange in self.axis.keys() if exchange in ["binance", "okx", "bybit", "gateio"] for flow in self.axis[exchange].perpetual_axis["liquidations"].keys() 
            }
                                           )

        self.positionsTTA =  synthesis.indomnifier("btc", "perpetul", "TTA",  {
                "_".join([exchange, flow]) : self.axis[exchange].perpetual_axis["indicators"][flow] for exchange in self.axis.keys() if exchange in ["binance"] for flow in self.axis[exchange].perpetual_axis["indicators"].keys() if "TTA" in flow
            },
            self.oifunding
                                           )

        self.positionsTTP =  synthesis.indomnifier("btc", "perpetul", "TTP", {
                "_".join([exchange, flow]) : self.axis[exchange].perpetual_axis["indicators"][flow] for exchange in self.axis.keys() if exchange in ["binance"] for flow in self.axis[exchange].perpetual_axis["indicators"].keys() if "TTP" in flow
            },
            self.oifunding
                                           )

        self.positionsGTA =  synthesis.indomnifier("btc", "perpetul", "GTA", {
                "_".join([exchange, flow]) : self.axis[exchange].perpetual_axis["indicators"][flow] for exchange in self.axis.keys() if exchange in ["binance", "okx", "bybit"] for flow in self.axis[exchange].perpetual_axis["indicators"].keys() if "GTA" in flow
            },
            self.oifunding
                                           )
        self.data = dict()


    def merge(self):
        for insType in self.books.keys():
            try:
                self.books[insType].merge_snapshots()
                self.data["_".join(["books", insType])] = self.books[insType].snapshotO
            except:
                pass
        for insType in self.trades.keys():
            try:
                self.trades[insType].merge_snapshots()
                self.data["_".join(["trades", insType])] = self.trades[insType].snapshotO
            except:
                pass
        for insType in self.adjustmests.keys():
            try:
                self.adjustmests[insType].get_adjusted_orders()
                self.data["_".join(["trades", insType])] = self.adjustmests[insType].data
            except:
                pass
        try:
            self.oifunding.merge_snapshots()
            self.data["_".join(["oifunding", "perpetual"])] = self.oifunding.snapshot
        except:
            pass
        try:
            self.liquidations.merge_snapshots()
            self.data["_".join(["liquidations", "perpetual"])] = self.liquidations.snapshot
        except:
            pass
        try:
            self.positionsTTA.merge()
            self.data["_".join(["TTA", "perpetual"])] = self.positionsTTA.data
        except:
            pass
        try:
            self.positionsTTP.merge()
            self.data["_".join(["TTP", "perpetual"])] = self.positionsTTA.data
        except:
            pass
        try:
            self.positionsGTA.merge()
            self.data["_".join(["GTA", "perpetual"])] = self.positionsTTA.data
        except:
            pass
        
    # Merge
    def ratrive_data(self, data=None, type_=None):
        """
            books_spot, books_perp, trades_spot, trades_perp, oifunding, liquidations, TTA, TTP, GTA
        """
        if data == None and type_ == None:
            return self.data
    
    # add data

    # Spot books

    def add_binance_spot_btcusdt_depth(self, data):
        self.books["spot"].axis["binance_btcusdt"].update_books(data)

    def add_binance_spot_btcfdusd_depth(self, data):
        self.books["spot"].axis["binance_btcfdusd"].update_books(data)

    def add_okx_spot_btcusdt_depth(self, data):
        self.books["spot"].axis["okx_btcusdt"].update_books(data)

    def add_bybit_spot_btcusdt_depth(self, data):
        self.books["spot"].axis["bybit_btcusdt"].update_books(data)

    def add_bybit_spot_btcusdt_depth(self, data):
        self.books["spot"].axis["bybit_btcusdt"].update_books(data)

    def add_bybit_spot_btcusdc_depth(self, data):
        self.books["spot"].axis["bybit_btcusdc"].update_books(data)

    def add_bingx_spot_btcusdt_depth(self, data):
        self.books["spot"].axis["bingx_btcusdt"].update_books(data)

    def add_mexc_spot_btcusdt_depth(self, data):
        self.books["spot"].axis["mexc_btcusdt"].update_books(data)

    def add_kucoin_spot_btcusdt_depth(self, data):
        self.books["spot"].axis["kucoin_btcusdt"].update_books(data)

    def add_coinbase_spot_btcusd_depth(self, data):
        self.books["spot"].axis["coinbase_btcusd"].update_books(data)

    def add_htx_spot_btcusdt_depth(self, data):
        self.books["spot"].axis["htx_btcusdt"].update_books(data)

    def add_gateio_spot_btcusdt_depth(self, data):
        self.books["spot"].axis["gateio_btcusdt"].update_books(data)

    def add_bitget_spot_btcusdt_depth(self, data):
        self.books["spot"].axis["bitget_btcusdt"].update_books(data)

    # Spot trades

    def add_binance_spot_btcusdt_trades(self, data):
        self.trades["spot"].axis["binance_btcusdt"].input_trades(data)

    def add_binance_spot_btcfdusd_trades(self, data):
        self.trades["spot"].axis["binance_btcfdusd"].input_trades(data)

    def add_okx_spot_btcusdt_trades(self, data):
        self.trades["spot"].axis["okx_btcusdt"].input_trades(data)

    def add_bybit_spot_btcusdt_trades(self, data):
        self.trades["spot"].axis["bybit_btcusdt"].input_trades(data)

    def add_bybit_spot_btcusdt_trades(self, data):
        self.trades["spot"].axis["bybit_btcusdt"].input_trades(data)

    def add_bybit_spot_btcusdc_trades(self, data):
        self.trades["spot"].axis["bybit_btcusdc"].input_trades(data)

    def add_bingx_spot_btcusdt_trades(self, data):
        self.trades["spot"].axis["bingx_btcusdt"].input_trades(data)

    def add_mexc_spot_btcusdt_trades(self, data):
        self.trades["spot"].axis["mexc_btcusdt"].input_trades(data)

    def add_kucoin_spot_btcusdt_trades(self, data):
        self.trades["spot"].axis["kucoin_btcusdt"].input_trades(data)

    def add_coinbase_spot_btcusd_trades(self, data):
        self.trades["spot"].axis["coinbase_btcusd"].input_trades(data)

    def add_htx_spot_btcusdt_trades(self, data):
        self.trades["spot"].axis["htx_btcusdt"].input_trades(data)

    def add_gateio_spot_btcusdt_trades(self, data):
        self.trades["spot"].axis["gateio_btcusdt"].input_trades(data)

    def add_bitget_spot_btcusdt_trades(self, data):
        self.trades["spot"].axis["bitget_btcusdt"].input_trades(data)

    # Perp books
    
    def add_binance_perp_btcusdt_depth(self, data):
        self.books["perpetual"].axis["binance_btcusdt"].update_books(data) 

    def add_binance_perp_btcusd_depth(self, data):
        self.books["perpetual"].axis["binance_btcusd"].update_books(data) 

    def add_okx_perp_btcusdt_depth(self, data):
        self.books["perpetual"].axis["okx_btcusdt"].update_books(data) 

    def add_okx_perp_btcusd_depth(self, data):
        self.books["perpetual"].axis["okx_btcusd"].update_books(data)

    def add_bybit_perp_btcusdt_depth(self, data):
        self.books["perpetual"].axis["bybit_btcusdt"].update_books(data) 

    def add_bybit_perp_btcusd_depth(self, data):
        self.books["perpetual"].axis["bybit_btcusd"].update_books(data)

    def add_bingx_perp_btcusdt_depth(self, data):
        self.books["perpetual"].axis["bingx_btcusdt"].update_books(data)   

    def add_mexc_perp_btcusdt_depth(self, data):
        self.books["perpetual"].axis["mexc_btcusdt"].update_books(data) 

    def add_kucoin_perp_btcusdt_depth(self, data):
        self.books["perpetual"].axis["kucoin_btcusdt"].update_books(data) 

    def add_deribit_perp_btcusd_depth(self, data):
        self.books["perpetual"].axis["deribit_btcusd"].update_books(data) 

    def add_htx_perp_btcusdt_depth(self, data):
        self.books["perpetual"].axis["htx_btcusdt"].update_books(data) 

    def add_gateio_perp_btcusdt_depth(self, data):
        self.books["perpetual"].axis["gateio_btcusdt"].update_books(data) 

    def add_bitget_perp_btcusdt_depth(self, data):
        self.books["perpetual"].axis["bitget_btcusdt"].update_books(data) 

    # Perp trades

    def add_binance_perp_btcusdt_trades(self, data):
        self.trades["perpetual"].axis["binance_btcusdt"].input_trades(data) 

    def add_binance_perp_btcusd_trades(self, data):
        self.trades["perpetual"].axis["binance_btcusd"].input_trades(data) 

    def add_okx_perp_btcusdt_trades(self, data):
        self.trades["perpetual"].axis["okx_btcusdt"].input_trades(data) 

    def add_okx_perp_btcusd_trades(self, data):
        self.trades["perpetual"].axis["okx_btcusd"].input_trades(data)

    def add_bybit_perp_btcusdt_trades(self, data):
        self.trades["perpetual"].axis["bybit_btcusdt"].input_trades(data) 

    def add_bybit_perp_btcusd_trades(self, data):
        self.trades["perpetual"].axis["bybit_btcusd"].input_trades(data)

    def add_bingx_perp_btcusdt_trades(self, data):
        self.trades["perpetual"].axis["bingx_btcusdt"].input_trades(data)   

    def add_mexc_perp_btcusdt_trades(self, data):
        self.trades["perpetual"].axis["mexc_btcusdt"].input_trades(data) 

    def add_kucoin_perp_btcusdt_trades(self, data):
        self.trades["perpetual"].axis["kucoin_btcusdt"].input_trades(data) 

    def add_deribit_perp_btcusd_trades(self, data):
        self.trades["perpetual"].axis["deribit_btcusd"].input_trades(data) 

    def add_htx_perp_btcusdt_trades(self, data):
        self.trades["perpetual"].axis["htx_btcusdt"].input_trades(data) 

    def add_gateio_perp_btcusdt_trades(self, data):
        self.trades["perpetual"].axis["gateio_btcusdt"].input_trades(data) 

    def add_bitget_perp_btcusdt_trades(self, data):
        self.trades["perpetual"].axis["bitget_btcusdt"].input_trades(data)         

    # OI

    def add_binance_perp_btcusdt_oi(self, data):
        self.oifunding.axis["binance_btcusdt"].input_oi(data) 

    def add_binance_perp_btcusd_oi(self, data):
        self.oifunding.axis["binance_btcusd"].input_oi(data) 

    def add_okx_perp_btcusdt_oi(self, data):
        self.oifunding.axis["okx_btcusdt"].input_oi(data) 

    def add_okx_perp_btcusd_oi(self, data):
        self.oifunding.axis["okx_btcusd"].input_oi(data) 

    def add_bingx_perp_btcusdt_oi(self, data):
        self.oifunding.axis["bingx_btcusdt"].input_oi(data) 

    def add_gateio_perp_btcusdt_oi(self, data):
        self.oifunding.axis["gateio_btcusdt"].input_oi(data) 

    def add_htx_perp_btcusdt_oi(self, data):
        self.oifunding.axis["htx_btcusdt"].input_oi(data) 
    
    # Funding

    def add_binance_perp_btcusdt_funding(self, data):
        self.oifunding.axis["binance_btcusdt"].input_funding(data) 

    def add_binance_perp_btcusd_funding(self, data):
        self.oifunding.axis["binance_btcusd"].input_funding(data) 

    def add_okx_perp_btcusdt_funding(self, data):
        self.oifunding.axis["okx_btcusdt"].input_funding(data) 

    def add_okx_perp_btcusd_funding(self, data):
        self.oifunding.axis["okx_btcusd"].input_funding(data) 

    def add_bingx_perp_btcusdt_funding(self, data):
        self.oifunding.axis["bingx_btcusdt"].input_funding(data) 

    def add_gateio_perp_btcusdt_funding(self, data):
        self.oifunding.axis["gateio_btcusdt"].input_funding(data) 

    def add_htx_perp_btcusdt_funding(self, data):
        self.oifunding.axis["htx_btcusdt"].input_funding(data) 
    
    # OIFund  

    def add_bybit_perp_btcusdt_oifunding(self, data):
        self.oifunding.axis["bybit_btcusdt"].input_oi_funding(data) 

    def add_bybit_perp_btcusd_oifunding(self, data):
        self.oifunding.axis["bybit_btcusd"].input_oi_funding(data) 

    def add_mexc_perp_btcusdt_oifunding(self, data):
        self.oifunding.axis["mexc_btcusdt"].input_oi_funding(data) 

    def add_kucoin_perp_btcusdt_oifunding(self, data):
        self.oifunding.axis["kucoin_btcusdt"].input_oi_funding(data) 

    def add_bitget_perp_btcusdt_oifunding(self, data):
        self.oifunding.axis["bitget_btcusdt"].input_oi_funding(data) 

    def add_htx_perp_btcusdt_funding(self, data):
        self.oifunding.axis["htx_btcusdt"].input_oi_funding(data) 

    def add_deribit_perp_btcusd_oifunding(self, data):
        self.oifunding.axis["deribit_btcusd"].input_oi_funding(data) 
    
    # Liquidations
    
    def add_binance_perp_btcusdt_liquidations(self, data):
        self.liquidations.axis["binance_btcusdt"].input_liquidations(data) 

    def add_binance_perp_btcusd_liquidations(self, data):
        self.liquidations.axis["binance_btcusd"].input_liquidations(data) 

    def add_okx_perp_btc_liquidations(self, data):
        self.liquidations.axis["okx_btc"].input_liquidations(data) 

    def add_bybit_perp_btcusdt_liquidations(self, data):
        self.liquidations.axis["bybit_btcusdt"].input_liquidations(data) 

    def add_bybit_perp_btcusd_liquidations(self, data):
        self.liquidations.axis["bybit_btcusd"].input_liquidations(data)

    def add_gateio_perp_btcusdt_liquidations(self, data):
        self.liquidations.axis["gateio_btcusdt"].input_liquidations(data)  
    
    # Positions

    def add_binance_perp_btcusdt_tta(self, data):
        self.positionsTTA.axis_ratio["binance_btcusdt_TTA"].input_binance_gta_tta_ttp(data) 

    def add_binance_perp_btcusd_tta(self, data):
        self.positionsTTA.axis_ratio["binance_btcusd_TTA"].input_binance_gta_tta_ttp(data) 

    def add_binance_perp_btcusdt_ttp(self, data):
        self.positionsTTP.axis_ratio["binance_btcusdt_TTP"].input_binance_gta_tta_ttp(data) 

    def add_binance_perp_btcusd_ttp(self, data):
        self.positionsTTP.axis_ratio["binance_btcusd_TTP"].input_binance_gta_tta_ttp(data) 

    def add_binance_perp_btcusdt_gta(self, data):
        self.positionsGTA.axis_ratio["binance_btcusdt_GTA"].input_binance_gta_tta_ttp(data) 

    def add_binance_perp_btcusd_gta(self, data):
        self.positionsGTA.axis_ratio["binance_btcusd_GTA"].input_binance_gta_tta_ttp(data) 

    def add_okx_perp_btc_gta(self, data):
        self.positionsGTA.axis_ratio["okx_GTA"].input_okx_gta(data) 

    def add_bybit_perp_btcusdt_gta(self, data):
        self.positionsGTA.axis_ratio["bybit_btcusdt_GTA"].input_bybit_gta(data) 

    def add_bybit_perp_btcusd_gta(self, data):
        self.positionsGTA.axis_ratio["bybit_btcusd_GTA"].input_bybit_gta(data) 

    # Test Synth

    def input_from_json(self):
        """
            child directory data should contain json files. if the methods is add_bybit_perp_btcusd_gta, then the file should be names bybit_perp_btcusd_gta.json
        """
        extra_methods = ["ratrive_data", "merge", "input_from_json", "test_empty_dataframes", "test_display_books", "test_display_trades", "test_display_oif", "test_display_liquidations", "test_display_gta", "test_display_tta", "test_display_ttp"]
        callable_methods = [method for method in dir(self) if callable(getattr(self, method)) and not method.startswith("__")]
        methods = []
        methods_str = []
        for method_name in callable_methods:
            if method_name not in extra_methods:
                method = getattr(self, method_name)
                methods.append(method)
                methods_str.append(method_name)    
        import os
        parent_dir = os.getcwd()
        data_dir  = parent_dir + '/data'
        files = [f for f in os.listdir(data_dir) if os.path.isfile(os.path.join(data_dir, f))]
        methods_str = ["_".join(x.split("_")[1:]) + ".json" for x in sorted(methods_str)]
        files = [x for x in sorted(files) if x not in ["bybit_option_btc_oi.json", "okx_option_btc_oi.json", "deribit_option_btc_oi.json", "news.json"]]

        v = {
            x : { 
            "data" : json.load(open("".join(["data/", x]))),
            "lookup" : f,
            } 
            for x, f in zip(files, methods)
            }

        for key in v.keys():
            for d in v[key]["data"]:
                d = json.dumps(d)
                v[key]["lookup"](d)

    def test_empty_dataframes(self):

        total_empty = []

        # spot

        for books, trades in zip(self.books["spot"].axis, self.trades["spot"].axis):
            b = self.books["spot"].axis[books].snapshot
            t = self.trades["spot"].axis[trades].snapshot_total
            try:
                if b.empty:
                    print(f"spot, books, {books}, is empty")
                    total_empty.append(b)
            except:
                print(f"spot, books, {books}, is empty")
                total_empty.append(b)
            try:
                if t.empty:
                    print(f"spot, trades, {trades}, is empty")
                    total_empty.append(b)
            except:
                print(f"spot, trades, {trades}, is empty")
                total_empty.append(b)
        
        # perp

        for books, trades in zip(self.books["perpetual"].axis, self.trades["perpetual"].axis):
            b = self.books["perpetual"].axis[books].snapshot
            t = self.trades["perpetual"].axis[trades].snapshot_total
            try:
                if b.empty:
                    print(f"perpetual, books, {books}, is empty")
                    total_empty.append(b)
            except:
                print(f"perpetual, books, {books}, is empty")
                total_empty.append(b)
            try:
                if t.empty:
                    print(f"perpetual, trades, {trades}, is empty")
                    total_empty.append(b)
            except:
                print(f"perpetual, trades, {trades}, is empty")
                total_empty.append(b)

        # oif

        for oiff in self.oifunding.axis:
            oif = self.oifunding.axis[oiff].snapshot
            try:
                if len(oif) != 0:
                    pass
                else:
                    print(f"perpetual, oif, {oiff}, is empty")
                    total_empty.append(p)
            except:
                print(f"perpetual, oif, {oiff}, is empty")
                total_empty.append(b)


        # Liquidations

        for liqss in self.liquidations.axis:
            liqs = self.liquidations.axis[liqss].snapshot_total
            try:
                if liqs.empty:
                    print(f"perpetual, liquidations, {liqss}, is empty")
                    total_empty.append(b)
            except:
                print(f"perpetual, liquidations, {liqss}, is empty")
                total_empty.append(b)

        # Positions
        
        for TTA in self.positionsTTA.axis:
            p = self.positionsTTA.axis[TTA].data
            if len(p) != 0:
                pass
            else:
                print(f"perpetual, TTA, {TTA}, is empty")
                total_empty.append(p)
        
        for TTP in self.positionsTTP.axis:
            p = self.positionsTTP.axis[TTP].data
            if len(p) != 0:
                pass
            else:
                print(f"perpetual, TTP, {TTP}, is empty")
                total_empty.append(p)

        for GTA in self.positionsGTA.axis:
            p = self.positionsGTA.axis[GTA].data
            if len(p) != 0:
                pass
            else:
                print(f"perpetual, GTA, {GTA}, is empty")
                total_empty.append(p)

        if len(total_empty) == 0:
            print("No empty frames")
        
        # return total_empty
    
    def test_display_books(self, insType, exinstrument):
        """
            insType : spot. perpetual
            exinstrument : Ex. gateio_btcusdt
        """
        print(self.books[insType].axis[exinstrument].df)

    def test_display_trades(self, insType, exinstrument):
        """
            insType : spot. perpetual
            exinstrument : Ex. gateio_btcusdt
        """
        print(self.trades[insType].axis[exinstrument].buys)
        if self.trades[insType].axis[exinstrument].buys.empty:
            print(self.trades[insType].axis[exinstrument].sells)

    def test_display_oif(self, exinstrument):
        """
            exinstrument : Ex. gateio_btcusdt
        """
        print(self.oifunding.axis[exinstrument].raw_data)

    def test_display_liquidations(self, exinstrument):
        """
            exinstrument : Ex. gateio_btcusdt
        """
        print(self.liquidations.axis[exinstrument].longs)
        if self.liquidations.axis[exinstrument].longs.empty:
            print(self.liquidations.axis[exinstrument].shorts)

    def test_display_gta(self, exinstrument):
        """
            exinstrument : Ex. binance_btcusdt
        """
        print(self.positionsGTA.axis[exinstrument].data)

    def test_display_tta(self, exinstrument):
        """
            exinstrument : Ex. binance_btcusdt
        """
        print(self.positionsGTA.axis[exinstrument].data)

    def test_display_ttp(self, exinstrument):
        """
            exinstrument : Ex. binance_btcusdt
        """
        print(self.positionsTTP.axis[exinstrument].data)

# From provided data files some had less than 60 seconds of data
        
# spot, trades, bingx_btcusdt, is empty                  # ok
# spot, books, mexc_btcusdt, is empty                    # ok
# spot, books, kucoin_btcusdt, is empty                  # ok
# spot, books, htx_btcusdt, is empty                    # ok
# spot, trades, htx_btcusdt, is empty                     # ok
# spot, books, bitget_btcusdt, is empty                   # ok
# perpetual, trades, bybit_btcusd, is empty               # ok
# perpetual, books, mexc_btcusdt, is empty                # ok
# perpetual, trades, mexc_btcusdt, is empty               # ok
# perpetual, books, kucoin_btcusdt, is empty              # ok
# perpetual, books, htx_btcusdt, is empty                 # ok    
# perpetual, trades, htx_btcusdt, is empty                # ok
# perpetual, books, gateio_btcusdt, is empty              # ok
# perpetual, trades, gateio_btcusdt, is empty             # ok  
# perpetual, oif, gateio_btcusdt, is empty                # ok
# perpetual, oif, bitget_btcusdt, is empty                # ok
# perpetual, liquidations, binance_btcusdt, is empty      # ok
# perpetual, liquidations, binance_btcusd, is empty       # ok
# perpetual, liquidations, okx_btc, is empty              # ok
# perpetual, liquidations, bybit_btcusdt, is empty        # ok
# perpetual, liquidations, bybit_btcusd, is empty         # ok
# perpetual, liquidations, gateio_btcusdt, is empty       # ok


In [3]:
import warnings
warnings.filterwarnings("ignore")
a = btcSynth(20)
a.input_from_json()

AttributeError: 'indomnifier' object has no attribute 'axis'

In [5]:
a.input_oi()

a.positionsGTA.merge()

In [7]:
a.data

{'liquidations_perpetual': {'timestamp': '2024-03-07 14:54',
  'price': 9910.0,
  'longsTotal': 15.453496247835291,
  'longs': {'120.0': 15.39349624783529, '3440.0': 0.060000000000000005},
  'shortsTotal': 0.06921903347302531,
  'shorts': {'9920.0': 0.042,
   '9440.0': 0.021219033473025305,
   '43520.0': 0.006},
  'longsLiquidations': [15.39349624783529, 0.060000000000000005],
  'shortsLiquidations': [0.014,
   0.014,
   0.014,
   0.010609516736512652,
   0.010609516736512652,
   0.003,
   0.003]}}

In [ ]:
b = a.test_empty_dataframes()

In [ ]:
a.liquidations.axis["gateio_btcusdt"].longs

In [16]:
class indomnifier():

    """
        Merges the same indicator from different exchanges weighted by total open interest
    """

    def __init__ (self, instrument, insType, indType):
        """
            axis : the dictionary of classes indicatorsflow
        """
        self.instrument = instrument
        self.insType = insType
        self.indType = indType
        self.data = dict()
        self.open_interest = dict()
        self.ratios = dict()

    def retrive_data(self, key=None):
        """
            timestamp, ratio
            is key == None, returns the whole dictionary
        """
        if key != None:
            return self.data.get(key)
        else:
            return self.data

    def input_oi(self, exchange, instrument, data):
        """
            Inputs open interest by instrument 
        """
        self.open_interest["_".join([exchange, instrument])] = data

    def input_ratio(self, exchange, instrument, data):
        """
            Inputs open interest by instrument 
        """
        self.ratios["_".join([exchange, instrument])] = data


    def merge(self):
        """
            merges gta, gtp, tta, ttp's
            since okx provides global btc long shorts ration, the code has some slight changes
        """
        r = []
        o = []
        for key in set(self.open_interest.keys()).union(self.ratios.keys()):
            ratio = self.ratios.get(key, 0)
            oi = self.open_interest.get(key, 0)
            r.append(ratio)
            o.append(oi)

        o.append(self.open_interest.get("okx_btcusdt", 0) + self.open_interest.get("okx_btcusd", 0))
        r.append(self.open_interest.get("okx_btc", 0))
        weighted_ratio = np.average(r, weights=o)
        self.data["ratio"] = weighted_ratio
        self.data["timestamp"] = datetime.datetime.now().strftime("%Y-%m-%d %H:%M")

In [20]:
a = indomnifier("a", "a", "a")
a.input_oi("binance", "btcusdt", 1)
a.input_oi("okx", "btcusdt", 1)
a.input_oi("okx", "btcusd", 5)

a.input_ratio("binance", "btcusdt", 0.1)
a.input_ratio("okx", "btc", 0.6)

In [21]:
a.merge()

In [22]:
a.data

{'ratio': 0.007692307692307693, 'timestamp': '2024-03-07 16:07'}